<a href="https://colab.research.google.com/github/wondifraw/10Acadamy/blob/main/Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telethon 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 19.1 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=732a3bbc7e797ca7c2600e0d396db40baf3051fe56c475352e7de1c7c66ce030
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [2]:
!pip install telethon python-dotenv

In [22]:
from telegram_scraper import TelegramScraper
import asyncio

# Telegram API
api_id = '27932004'
api_hash = ''

scraper = TelegramScraper(api_id, api_hash)
# Function to fetch messages from all channels and save as CSV
async def fetch_and_save_all(scraper, limit=1000):
    all_data = []
    for channel in scraper.get_channel_list():
        data = await scraper.fetch_messages(channel, limit=limit)
        all_data.extend(data)
    scraper.save_data_csv(all_data, 'data/raw/telegram_messages.csv')
    print('All messages saved to data/raw/telegram_messages.csv')
# Run the scraping and saving
await fetch_and_save_all(scraper, limit=1000)

Connecting to Telegram...
Fetching channel entity for @Leyueqa...
Fetching messages from @Leyueqa...
Successfully fetched 1000 messages from @Leyueqa.
Connecting to Telegram...
Fetching channel entity for @sinayelj...
Fetching messages from @sinayelj...
Successfully fetched 1000 messages from @sinayelj.
Connecting to Telegram...
Fetching channel entity for @Shewabrand...
Fetching messages from @Shewabrand...
Successfully fetched 1000 messages from @Shewabrand.
Connecting to Telegram...
Fetching channel entity for @helloomarketethiopia...
Fetching messages from @helloomarketethiopia...
Successfully fetched 1000 messages from @helloomarketethiopia.
Connecting to Telegram...
Fetching channel entity for @modernshoppingcenter...
Fetching messages from @modernshoppingcenter...
Successfully fetched 1000 messages from @modernshoppingcenter.
Data successfully saved to data/raw/telegram_messages.csv
All messages saved to data/raw/telegram_messages.csv


In [ ]:
import pandas as pd
from preprocessing.amharic_text import *

input_path = '../data/raw/telegram_raw_messages.csv'
output_path = '../data/processed/telegram_cleaned_messages.csv'

# Load data
df = pd.read_csv(input_path)

# Ensure column is string and apply preprocessing
df['Message'] = df['Message'].astype(str).apply(preprocess_amharic_message)

# Remove empty messages (after preprocessing) and drop duplicates
df = df[df['Message'].str.strip().astype(bool)]  # keep non-empty
df = df.drop_duplicates(subset=['Message'])      # drop duplicate messages

# Show sample
print(df[['Channel Username', 'Message']].head())

# Save result
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"✅ Preprocessed Amharic messages (cleaned & deduplicated) saved to: {output_path}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/clean_telegram_data.csv'